<a href="https://colab.research.google.com/github/duke-sunshine/NYU_Econometrics_Summer2021/blob/main/Economics_Milestone4_IV2SLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Briefings**

1. Two packages:


*   https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html
*   https://bashtage.github.io/linearmodels/doc/iv/examples/advanced-examples.html

2. Four variables:

Dependent Variables: Y
Endogenous Variable: X
Control variables: W
instruments: Z

3. Two regressions


*   First: X~Z
*   Second: Y~(estimated_X+W)





background readings: https://cryptobriefing.com/earning-20-apy-with-yield-app/

# Data Preparations

### Date Sources 
https://docs.defipulse.com/overview-of-api-endpoints

https://data-beta.defipulse.com/dashboard/egs

https://docs.defipulse.com/api-docs-by-provider/defi-pulse-data/total-value-locked/total-value-locked

### Import the Historical Data of Total Value Locked

In [ ]:
import pandas as pd
import requests
import json 
from datetime import datetime
TVL= requests.get('https://data-api.defipulse.com/api/v1/defipulse/api/GetHistory?resolution=history&project=all&api-key=5eafa87608af1f342443fc1fc293d5a596f3aa509526c9c0f663a1b5723a')
TVL= TVL.json()
TVL=pd.DataFrame(TVL)
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI
0,1623744000,63193338166,2.418254e+07,175598.975642,8.776455e+06,2.954992e+09
1,1623715200,62052366020,2.403807e+07,175064.886165,8.796396e+06,2.954793e+09
2,1623628800,60877945349,2.425010e+07,174891.430037,8.850787e+06,2.958830e+09
3,1623542400,57779398925,2.436171e+07,176037.991729,8.856534e+06,2.906475e+09
4,1623456000,57892020781,2.459837e+07,174001.967355,8.943316e+06,2.890921e+09


### Epoch & Unix Timestamp Conversion Tools: 

https://www.epochconverter.com/

In [ ]:
import datetime
TVL['timestamp']=TVL['timestamp'].astype(int)
TVL.dtypes

timestamp      int64
tvlUSD         int64
tvlETH       float64
BTC          float64
ETH          float64
DAI          float64
dtype: object

In [ ]:
TVL['time']=TVL['timestamp'].apply(lambda x: datetime.datetime.utcfromtimestamp(x).replace(tzinfo=datetime.timezone.utc))
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time
0,1623744000,63193338166,2.418254e+07,175598.975642,8.776455e+06,2.954992e+09,2021-06-15 08:00:00+00:00
1,1623715200,62052366020,2.403807e+07,175064.886165,8.796396e+06,2.954793e+09,2021-06-15 00:00:00+00:00
2,1623628800,60877945349,2.425010e+07,174891.430037,8.850787e+06,2.958830e+09,2021-06-14 00:00:00+00:00
3,1623542400,57779398925,2.436171e+07,176037.991729,8.856534e+06,2.906475e+09,2021-06-13 00:00:00+00:00
4,1623456000,57892020781,2.459837e+07,174001.967355,8.943316e+06,2.890921e+09,2021-06-12 00:00:00+00:00


In [ ]:
TVL['date'] = pd.to_datetime(TVL['time']).dt.date
TVL.head()

,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,time,date
0,1623744000,63193338166,2.418254e+07,175598.975642,8.776455e+06,2.954992e+09,2021-06-15 08:00:00+00:00,2021-06-15
1,1623715200,62052366020,2.403807e+07,175064.886165,8.796396e+06,2.954793e+09,2021-06-15 00:00:00+00:00,2021-06-15
2,1623628800,60877945349,2.425010e+07,174891.430037,8.850787e+06,2.958830e+09,2021-06-14 00:00:00+00:00,2021-06-14
3,1623542400,57779398925,2.436171e+07,176037.991729,8.856534e+06,2.906475e+09,2021-06-13 00:00:00+00:00,2021-06-13
4,1623456000,57892020781,2.459837e+07,174001.967355,8.943316e+06,2.890921e+09,2021-06-12 00:00:00+00:00,2021-06-12


In [ ]:
TVL=TVL.drop(columns='time')
TVL['date']=pd.to_datetime(TVL['date'])
TVL.dtypes

timestamp             int64
tvlUSD                int64
tvlETH              float64
BTC                 float64
ETH                 float64
DAI                 float64
date         datetime64[ns]
dtype: object

### Import the data for Ethereum Prices

[Data Metrics](https://github.com/coinmetrics-io/data/blob/master/csv/metrics.csv)

In [ ]:
df_eth=pd.read_csv("https://raw.githubusercontent.com/coinmetrics-io/data/master/csv/eth.csv",usecols=(["time","PriceUSD","CapMrktCurUSD","TxTfrValAdjUSD","AdrBalCnt","NVTAdj90","SplyCur"]))
df_eth['time']=pd.to_datetime(df_eth['time'])
df_eth.dtypes

time              datetime64[ns]
AdrBalCnt                float64
CapMrktCurUSD            float64
NVTAdj90                 float64
PriceUSD                 float64
SplyCur                  float64
TxTfrValAdjUSD           float64
dtype: object

### Merge the two Data Frame

In [ ]:
df =df_eth.merge(TVL, left_on='time', right_on='date')
df.head()

,time,AdrBalCnt,CapMrktCurUSD,NVTAdj90,PriceUSD,SplyCur,TxTfrValAdjUSD,timestamp,tvlUSD,tvlETH,BTC,ETH,DAI,date
0,2017-08-02,2563959.0,2.039684e+10,20.362229,217.584664,9.374206e+07,6.780871e+08,1501632000,0,0.000000,NaN,0.0,0.0,2017-08-02
1,2017-08-03,2583277.0,2.100563e+10,20.979627,224.028898,9.376303e+07,6.062566e+08,1501718400,4,0.018339,NaN,0.0,0.0,2017-08-03
2,2017-08-04,2605667.0,2.077052e+10,20.714810,221.471094,9.378433e+07,5.206240e+08,1501804800,4,0.017826,NaN,0.0,0.0,2017-08-04
3,2017-08-05,2624386.0,2.335192e+10,23.205816,248.939666,9.380552e+07,6.665590e+08,1501891200,5,0.022665,NaN,0.0,0.0,2017-08-05
4,2017-08-06,2640032.0,2.496085e+10,24.667311,266.031146,9.382679e+07,9.229966e+08,1501977600,5,0.019756,NaN,0.0,0.0,2017-08-06


In [ ]:
df=df[["date","tvlUSD","PriceUSD","SplyCur","AdrBalCnt"]]

# Test Instrument Validities

Y: "tvlUSD"

X: "PriceUSD"

Z: "SplyCur"

W: "AdrBalCnt"

In [ ]:
df[["PriceUSD","SplyCur"]].corr()

,PriceUSD,SplyCur
PriceUSD,1.000000,0.351985
SplyCur,0.351985,1.000000


# Using the Linear Model Package

https://bashtage.github.io/linearmodels/doc/iv/index.html

In [ ]:
pip install linearmodels

In [ ]:
from linearmodels import IV2SLS

### run an OLS

In [ ]:
df['const'] = 1

In [ ]:
ivolsmod = IV2SLS(df.tvlUSD, df[["const","PriceUSD","SplyCur","AdrBalCnt"]], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                 tvlUSD   R-squared:                      0.9436
Estimator:                        OLS   Adj. R-squared:                 0.9435
No. Observations:                1412   F-statistic:                 1.468e+04
Date:                Tue, Jun 15 2021   P-value (F-stat)                0.0000
Time:                        09:20:34   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const      -1.873e+10  1.114e+10    -1.6815     0.0927  -4.056e+10   3.101e+09
PriceUSD    1.995e+07  3.227e+05     61.816     0.00

### run an IV2SLS

In [ ]:
ivmod = IV2SLS(df.tvlUSD, df[["const","AdrBalCnt"]], df["PriceUSD"], df["SplyCur"])
res_2sls = ivmod.fit()
print(res_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 tvlUSD   R-squared:                      0.9436
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9435
No. Observations:                1412   F-statistic:                 1.426e+04
Date:                Tue, Jun 15 2021   P-value (F-stat)                0.0000
Time:                        09:20:34   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const       -1.28e+10  2.942e+08    -43.517     0.0000  -1.338e+10  -1.223e+10
AdrBalCnt      316.17     10.481     30.165     0.00

### comparison of the results

In [ ]:
from collections import OrderedDict
from linearmodels.iv.results import compare

In [ ]:
res = OrderedDict()
res['OLS'] = res_ols
res['2SLS'] = res_2sls
print(compare(res))

                 Model Comparison                
                                OLS          2SLS
-------------------------------------------------
Dep. Variable                tvlUSD        tvlUSD
Estimator                       OLS       IV-2SLS
No. Observations               1412          1412
Cov. Est.                    robust        robust
R-squared                    0.9436        0.9436
Adj. R-squared               0.9435        0.9435
F-statistic               1.468e+04     1.426e+04
P-value (F-stat)             0.0000        0.0000
==================     ============   ===========
const                    -1.873e+10     -1.28e+10
                          (-1.6815)     (-43.517)
PriceUSD                  1.995e+07     1.979e+07
                           (61.816)      (71.902)
SplyCur                      62.707              
                           (0.5358)              
AdrBalCnt                    288.69        316.17
                           (5.9462)      (30.165)


### Testing endogeneity

The Durbin test is a classic of endogeneity which compares OLS estimates with 2SLS and exploits the fact that OLS estimates will be relatively efficient. Durbin’s test is not robust to heteroskedasticity.

In [ ]:
res_2sls.durbin()

Durbin test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 0.1977
P-value: 0.6566
Distributed: chi2(1)
WaldTestStatistic, id: 0x7ff5bb4dcf90

Wooldridge’s regression-based test of exogeneity is robust to heteroskedasticity since it inherits the covariance estimator from the model. Here there is little difference.

In [ ]:
res_2sls.wooldridge_regression

Wooldridge's regression test of exogeneity
H0: Endogenous variables are exogenous
Statistic: 0.2870
P-value: 0.5921
Distributed: chi2(1)
WaldTestStatistic, id: 0x7ff5bb553850

### print first stage regression

In [ ]:
print(res_2sls.first_stage)

    First Stage Estimation Results    
                              PriceUSD
--------------------------------------
R-squared                       0.6445
Partial R-squared               0.5444
Shea's R-squared                0.5444
Partial F-statistic             1044.9
P-value (Partial F-stat)        0.0000
Partial F-stat Distn           chi2(1)
========================== ===========
const                        3.665e+04
                              (32.684)
AdrBalCnt                       0.0002
                              (31.562)
SplyCur                        -0.0004
                             (-32.325)
--------------------------------------

T-stats reported in parentheses
T-stats use same covariance type as original model


# Using the Statsmodel Package

Not recommended

https://www.statsmodels.org/dev/generated/statsmodels.sandbox.regression.gmm.IV2SLS.html